# Train a ConvNet using multiple GPUs

This notebook shows how to train a deep model using multiple GPUs. In this notebook, we use CIFAR-10 dataset and train ResNet using 2 GPUs.

First, **please make sure that you are running this notebook with an environment that has at least 2 GPUs**.

Let's import necessary packages:

In [ ]:
import numpy as np
import chainer
from chainer import cuda, Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.datasets import get_cifar10

# Dataset preparation

Chainer provides a utility to retrieve and handle the CIFAR-10/100 dataset. In this notebook, we use the dataset class provided by Chainer to use CIFAR-10.

In [ ]:
batchsize = 128

train, test = get_cifar10()
train_iter = iterators.MultiprocessIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

# Model definition

Let's define a deep model.

In [ ]:
class ResNet50(chainer.Chain):
    def __init__(self, n_class, n_blocks=[3, 4, 6, 3]):
        w = chainer.initializers.HeNormal()
        super(ResNet50, self).__init__(
            conv1=L.Convolution2D(
                None, 64, 7, 2, 3, initialW=w, nobias=True),
            bn1=L.BatchNormalization(64),
            res2=ResBlock(n_blocks[0], 64, 64, 256, 1),
            res3=ResBlock(n_blocks[1], 256, 128, 512),
            res4=ResBlock(n_blocks[2], 512, 256, 1024),
            res5=ResBlock(n_blocks[3], 1024, 512, 2048),
            fc6=L.Linear(2048, n_class))

    def __call__(self, x):
        h = self.bn1(self.conv1(x))
        h = F.max_pooling_2d(F.relu(h), 2, 2)
        h = self.res2(h)
        h = self.res3(h)
        h = self.res4(h)
        h = self.res5(h)
        h = F.average_pooling_2d(h, h.shape[2:], stride=1)
        h = self.fc6(h)
        if chainer.config.train:
            return h
        return F.softmax(h)


class ResBlock(chainer.ChainList):
    def __init__(self, n_layers, n_in, n_mid, n_out, stride=2):
        w = chainer.initializers.HeNormal()
        super(ResBlock, self).__init__()
        self.add_link(BottleNeck(n_in, n_mid, n_out, stride, True))
        for _ in range(n_layers - 1):
            self.add_link(BottleNeck(n_out, n_mid, n_out))

    def __call__(self, x):
        for f in self.children():
            x = f(x)
        return x


class BottleNeck(chainer.Chain):
    def __init__(self, n_in, n_mid, n_out, stride=1, proj=False):
        w = chainer.initializers.HeNormal()
        super(BottleNeck, self).__init__()
        with self.init_scope():
            self.conv1x1a = L.Convolution2D(
                n_in, n_mid, 1, stride, 0, initialW=w, nobias=True)
            self.conv3x3b = L.Convolution2D(
                n_mid, n_mid, 3, 1, 1, initialW=w, nobias=True)
            self.conv1x1c = L.Convolution2D(
                n_mid, n_out, 1, 1, 0, initialW=w, nobias=True)
            self.bn_a = L.BatchNormalization(n_mid)
            self.bn_b = L.BatchNormalization(n_mid)
            self.bn_c = L.BatchNormalization(n_out)
            if proj:
                self.conv1x1r = L.Convolution2D(
                    n_in, n_out, 1, stride, 0, initialW=w, nobias=True)
                self.bn_r = L.BatchNormalization(n_out)
        self.proj = proj

    def __call__(self, x):
        h = F.relu(self.bn_a(self.conv1x1a(x)))
        h = F.relu(self.bn_b(self.conv3x3b(h)))
        h = self.bn_c(self.conv1x1c(h))
        if self.proj:
            x = self.bn_r(self.conv1x1r(x))
        return F.relu(h + x)

Then instantiate the model and setup a optimizer.

In [ ]:
model = L.Classifier(ResNet50(n_class=10))
optimizer = optimizers.MomentumSGD(lr=0.01)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

# Create ParallelUpdater

To use multiple GPUs for training, use `ParallelUpdater` instead of `StarndardUpdater` for the updater given to a trainer. So what you need to do is just replacing `StandardUpdater` with `ParallelUpdater`. That's all!

In [ ]:
devices = {
    'main': 1,
    'gpu1': 2
}

updater = training.ParallelUpdater(
    train_iter, optimizer, devices=devices)

trainer = training.Trainer(
    updater, (100, 'epoch'), out='multi_gpu_result')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'val/main/loss', 'main/accuracy', 'val/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.Evaluator(test_iter, model, device=devices['main']), name='val')
trainer.extend(extensions.ExponentialShift('lr', 0.5), trigger=(25, 'epoch'))
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

trainer.run()

In [ ]:
from IPython.display import Image
Image('mnist_result/loss.png')
Image('mnist_result/accuracy.png')

# ChainerCV

[ChainerCV](https://github.com/chainer/chainercv) is a collection of tools to train and run neural networks for computer vision tasks using Chainer. It provides many kinds of models, their pre-trained weights, data augomentation algorithms, dataset abstraction, evaluation tools, etc.

In this notebook, let's use ChainerCV to apply some transformations to the training images for data augmentation.

In [ ]:
%%bash
pip install chainercv

Chainer's `TransformDataset` takes two arguments, the first one is a dataset object and the second one is a transformation function. Each datum in the dataset is transformed by using the transformation function. The function should take an input and perform some transformations using any kind of libraries or just array maniputation to the input ndarray, and then return the resulting transformed input.

Let's train the same model with the `TransformDataset` that apply random LR-flipping and random color augmentation using `chainercv.transforms.random_flip` and `chainercv.transforms.pca_lighting`, respectively.

In [ ]:
from chainercv import transforms

def transform(inputs):
    img, label = inputs
    img = transforms.random_flip(img)
    img = transforms.pca_lighting(img, 0.1)
    return img, label
    
batchsize = 128

train, test = get_cifar10()

train = datasets.TransformDataset(train, transform)

train_iter = iterators.MultiprocessIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)

model = L.Classifier(ResNet50(n_class=10))
optimizer = optimizers.MomentumSGD(lr=0.01)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer.WeightDecay(5e-4))

devices = {
    'main': 1,
    'gpu1': 2
}

updater = training.ParallelUpdater(
    train_iter, optimizer, devices=devices)

trainer = training.Trainer(
    updater, (100, 'epoch'), out='multi_gpu_result')

trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(['epoch', 'main/loss', 'val/main/loss', 'main/accuracy', 'val/main/accuracy', 'elapsed_time']))
trainer.extend(extensions.Evaluator(test_iter, model, device=devices['main']), name='val')
trainer.extend(extensions.ExponentialShift('lr', 0.5), trigger=(25, 'epoch'))
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))

trainer.run()